# Microsoft Agent Framework के साथ Human-in-the-Loop वर्कफ़्लो

## 🎯 सीखने के उद्देश्य

इस नोटबुक में, आप Microsoft Agent Framework के `RequestInfoExecutor` का उपयोग करके **human-in-the-loop** वर्कफ़्लो को लागू करना सीखेंगे। यह शक्तिशाली पैटर्न AI वर्कफ़्लो को रोकने और मानव इनपुट प्राप्त करने की अनुमति देता है, जिससे आपके एजेंट इंटरैक्टिव बनते हैं और महत्वपूर्ण निर्णयों पर मानव नियंत्रण प्रदान करते हैं।

## 🔄 Human-in-the-Loop क्या है?

**Human-in-the-loop (HITL)** एक डिज़ाइन पैटर्न है जिसमें AI एजेंट मानव इनपुट का अनुरोध करने के लिए निष्पादन को रोकते हैं और फिर आगे बढ़ते हैं। यह निम्नलिखित के लिए आवश्यक है:

- ✅ **महत्वपूर्ण निर्णय** - महत्वपूर्ण कार्यों को करने से पहले मानव अनुमोदन प्राप्त करें
- ✅ **अस्पष्ट स्थितियां** - जब AI अनिश्चित हो, तो मानव से स्पष्टता प्राप्त करें
- ✅ **उपयोगकर्ता प्राथमिकताएं** - उपयोगकर्ताओं को कई विकल्पों में से चुनने के लिए कहें
- ✅ **अनुपालन और सुरक्षा** - विनियमित संचालन के लिए मानव निगरानी सुनिश्चित करें
- ✅ **इंटरैक्टिव अनुभव** - ऐसे संवादात्मक एजेंट बनाएं जो उपयोगकर्ता इनपुट का उत्तर दें

## 🏗️ Microsoft Agent Framework में यह कैसे काम करता है

फ्रेमवर्क HITL के लिए तीन मुख्य घटक प्रदान करता है:

1. **`RequestInfoExecutor`** - एक विशेष एक्ज़ीक्यूटर जो वर्कफ़्लो को रोकता है और `RequestInfoEvent` को उत्सर्जित करता है
2. **`RequestInfoMessage`** - मानवों को भेजे गए टाइप किए गए अनुरोध पेलोड के लिए बेस क्लास
3. **`RequestResponse`** - `request_id` का उपयोग करके मानव प्रतिक्रियाओं को मूल अनुरोधों से जोड़ता है

**वर्कफ़्लो पैटर्न:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 हमारा उदाहरण: उपयोगकर्ता पुष्टि के साथ होटल बुकिंग

हम वैकल्पिक गंतव्यों का सुझाव देने से **पहले** मानव पुष्टि जोड़कर सशर्त वर्कफ़्लो पर निर्माण करेंगे:

1. उपयोगकर्ता एक गंतव्य का अनुरोध करता है (जैसे, "पेरिस")
2. `availability_agent` जांचता है कि कमरे उपलब्ध हैं या नहीं
3. **यदि कोई कमरे नहीं हैं** → `confirmation_agent` पूछता है "क्या आप विकल्प देखना चाहेंगे?"
4. वर्कफ़्लो **`RequestInfoExecutor` का उपयोग करके रुकता है**
5. **मानव प्रतिक्रिया देता है** "हाँ" या "नहीं" कंसोल इनपुट के माध्यम से
6. `decision_manager` प्रतिक्रिया के आधार पर मार्ग तय करता है:
   - **हाँ** → वैकल्पिक गंतव्य दिखाएं
   - **नहीं** → बुकिंग अनुरोध रद्द करें
7. अंतिम परिणाम प्रदर्शित करें

यह दिखाता है कि उपयोगकर्ताओं को एजेंट के सुझावों पर नियंत्रण कैसे दिया जा सकता है!

---

चलो शुरू करते हैं! 🚀


## चरण 1: आवश्यक लाइब्रेरी आयात करें

हम मानक एजेंट फ्रेमवर्क घटकों के साथ **मानव-इन-द-लूप विशिष्ट कक्षाओं** को आयात करते हैं:
- `RequestInfoExecutor` - ऐसा निष्पादक जो मानव इनपुट के लिए वर्कफ़्लो को रोकता है
- `RequestInfoEvent` - वह इवेंट जो मानव इनपुट की आवश्यकता होने पर उत्पन्न होता है
- `RequestInfoMessage` - टाइप किए गए अनुरोध पेलोड के लिए बेस क्लास
- `RequestResponse` - अनुरोधों के साथ मानव प्रतिक्रियाओं को जोड़ता है
- `WorkflowOutputEvent` - वर्कफ़्लो आउटपुट का पता लगाने के लिए इवेंट


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## चरण 2: संरचित आउटपुट के लिए Pydantic मॉडल परिभाषित करें

ये मॉडल **schema** को परिभाषित करते हैं जिसे एजेंट्स वापस करेंगे। हम सशर्त वर्कफ़्लो से सभी मॉडल रखते हैं और निम्नलिखित जोड़ते हैं:

**Human-in-the-Loop के लिए नया:**
- `HumanFeedbackRequest` - `RequestInfoMessage` का सबक्लास जो मनुष्यों को भेजे जाने वाले अनुरोध payload को परिभाषित करता है  
  - इसमें `prompt` (पूछने के लिए प्रश्न) और `destination` (अनुपलब्ध शहर के बारे में संदर्भ) शामिल है


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## चरण 3: होटल बुकिंग टूल बनाएं

वही टूल जो कंडीशनल वर्कफ़्लो से है - यह जांचता है कि गंतव्य में कमरे उपलब्ध हैं या नहीं।


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## चरण 4: रूटिंग के लिए कंडीशन फंक्शन्स परिभाषित करें

हमें अपने मानव-इन-द-लूप वर्कफ़्लो के लिए **चार कंडीशन फंक्शन्स** की आवश्यकता है:

**कंडीशनल वर्कफ़्लो से:**
1. `has_availability_condition` - रूट करता है जब होटल उपलब्ध हैं
2. `no_availability_condition` - रूट करता है जब होटल उपलब्ध नहीं हैं

**मानव-इन-द-लूप के लिए नया:**
3. `user_wants_alternatives_condition` - रूट करता है जब उपयोगकर्ता विकल्पों के लिए "हाँ" कहता है
4. `user_declines_alternatives_condition` - रूट करता है जब उपयोगकर्ता विकल्पों के लिए "नहीं" कहता है


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## चरण 5: निर्णय प्रबंधक निष्पादक बनाएं

यह **मानव-इन-द-लूप पैटर्न का मुख्य भाग** है! `DecisionManager` एक कस्टम `Executor` है जो:

1. **मानव प्रतिक्रिया प्राप्त करता है** `RequestResponse` ऑब्जेक्ट्स के माध्यम से
2. **उपयोगकर्ता के निर्णय को संसाधित करता है** (हाँ/नहीं)
3. **वर्कफ़्लो को मार्गित करता है** उपयुक्त एजेंट्स को संदेश भेजकर

मुख्य विशेषताएं:
- `@handler` डेकोरेटर का उपयोग करता है ताकि विधियों को वर्कफ़्लो चरणों के रूप में उजागर किया जा सके
- `RequestResponse[HumanFeedbackRequest, str]` प्राप्त करता है जिसमें मूल अनुरोध और उपयोगकर्ता का उत्तर दोनों शामिल होते हैं
- सरल "हाँ" या "नहीं" संदेश उत्पन्न करता है जो हमारी शर्त फ़ंक्शन्स को ट्रिगर करते हैं


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## चरण 6: कस्टम डिस्प्ले एक्ज़ीक्यूटर बनाएं

वही डिस्प्ले एक्ज़ीक्यूटर जो कंडीशनल वर्कफ़्लो से है - अंतिम परिणाम को वर्कफ़्लो आउटपुट के रूप में प्रदान करता है।


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## चरण 7: पर्यावरण वेरिएबल्स लोड करें

LLM क्लाइंट (GitHub Models, Azure OpenAI, या OpenAI) को कॉन्फ़िगर करें।


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## चरण 8: AI एजेंट और एक्सीक्यूटर बनाएं

हम **छह वर्कफ़्लो घटक** बनाते हैं:

**एजेंट (AgentExecutor में लिपटे हुए):**
1. **availability_agent** - टूल का उपयोग करके होटल की उपलब्धता की जांच करता है
2. **confirmation_agent** - 🆕 मानव पुष्टि अनुरोध तैयार करता है
3. **alternative_agent** - वैकल्पिक शहरों का सुझाव देता है (जब उपयोगकर्ता हां कहता है)
4. **booking_agent** - बुकिंग के लिए प्रोत्साहित करता है (जब कमरे उपलब्ध हों)
5. **cancellation_agent** - 🆕 रद्दीकरण संदेश संभालता है (जब उपयोगकर्ता नहीं कहता है)

**विशेष एक्सीक्यूटर:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor` जो मानव इनपुट के लिए वर्कफ़्लो को रोकता है
7. **decision_manager** - 🆕 कस्टम एक्सीक्यूटर जो मानव प्रतिक्रिया के आधार पर रूट करता है (पहले से ऊपर परिभाषित)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## चरण 9: मानव-इन-द-लूप के साथ वर्कफ़्लो बनाएं

अब हम **शर्तीय रूटिंग** सहित मानव-इन-द-लूप पथ के साथ वर्कफ़्लो ग्राफ़ बनाते हैं:

**वर्कफ़्लो संरचना:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**मुख्य किनारे:**
- `availability_agent → confirmation_agent` (जब कोई कमरे उपलब्ध नहीं हैं)
- `confirmation_agent → prepare_human_request` (प्रकार बदलें)
- `prepare_human_request → request_info_executor` (मानव के लिए विराम)
- `request_info_executor → decision_manager` (हमेशा - RequestResponse प्रदान करता है)
- `decision_manager → alternative_agent` (जब उपयोगकर्ता "हाँ" कहता है)
- `decision_manager → cancellation_agent` (जब उपयोगकर्ता "नहीं" कहता है)
- `availability_agent → booking_agent` (जब कमरे उपलब्ध हैं)
- सभी पथ `display_result` पर समाप्त होते हैं


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## चरण 10: टेस्ट केस 1 चलाएं - उपलब्धता के बिना शहर (पेरिस मानव पुष्टि के साथ)

यह परीक्षण **मानव-इन-द-लूप चक्र** को पूरी तरह से प्रदर्शित करता है:

1. पेरिस में होटल का अनुरोध करें
2. availability_agent जांच करता है → कोई कमरे उपलब्ध नहीं हैं
3. confirmation_agent मानव-सामना करने वाला प्रश्न बनाता है
4. request_info_executor **वर्कफ़्लो को रोकता है** और `RequestInfoEvent` उत्पन्न करता है
5. **एप्लिकेशन घटना का पता लगाता है और कंसोल में उपयोगकर्ता को संकेत देता है**
6. उपयोगकर्ता "हाँ" या "नहीं" टाइप करता है
7. एप्लिकेशन `send_responses_streaming()` के माध्यम से प्रतिक्रिया भेजता है
8. decision_manager प्रतिक्रिया के आधार पर मार्ग तय करता है
9. अंतिम परिणाम प्रदर्शित होता है

**मुख्य पैटर्न:**
- पहले चरण के लिए `workflow.run_stream()` का उपयोग करें
- अगले चरणों के लिए `workflow.send_responses_streaming(pending_responses)` का उपयोग करें
- जब मानव इनपुट की आवश्यकता हो तो `RequestInfoEvent` सुनें
- अंतिम परिणाम कैप्चर करने के लिए `WorkflowOutputEvent` सुनें


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## चरण 11: टेस्ट केस 2 चलाएं - उपलब्धता के साथ शहर (स्टॉकहोम - कोई मानव इनपुट आवश्यक नहीं)

यह परीक्षण दिखाता है कि जब कमरे उपलब्ध होते हैं तो **सीधा रास्ता** कैसे काम करता है:

1. स्टॉकहोम में होटल का अनुरोध करें
2. availability_agent जांच करता है → कमरे उपलब्ध ✅
3. booking_agent बुकिंग का सुझाव देता है
4. display_result पुष्टि दिखाता है
5. **कोई मानव इनपुट आवश्यक नहीं!**

जब कमरे उपलब्ध होते हैं, तो वर्कफ़्लो पूरी तरह से मानव-इन-द-लूप पथ को छोड़ देता है।


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## मुख्य बातें और मानव-इन-द-लूप सर्वोत्तम प्रथाएँ

### ✅ आपने क्या सीखा:

#### 1. **RequestInfoExecutor पैटर्न**
Microsoft Agent Framework में मानव-इन-द-लूप पैटर्न तीन मुख्य घटकों का उपयोग करता है:
- `RequestInfoExecutor` - वर्कफ़्लो को रोकता है और इवेंट्स उत्पन्न करता है
- `RequestInfoMessage` - टाइप किए गए अनुरोध पेलोड्स के लिए बेस क्लास (इसे सबक्लास करें!)
- `RequestResponse` - मानव प्रतिक्रियाओं को मूल अनुरोधों से जोड़ता है

**महत्वपूर्ण समझ:**
- `RequestInfoExecutor` स्वयं इनपुट एकत्र नहीं करता - यह केवल वर्कफ़्लो को रोकता है
- आपके एप्लिकेशन कोड को `RequestInfoEvent` सुनना चाहिए और इनपुट एकत्र करना चाहिए
- आपको `send_responses_streaming()` को कॉल करना होगा जिसमें `request_id` को उपयोगकर्ता के उत्तर से जोड़ने वाला डिक्शनरी हो

#### 2. **स्ट्रीमिंग निष्पादन पैटर्न**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **इवेंट-ड्रिवन आर्किटेक्चर**
वर्कफ़्लो को नियंत्रित करने के लिए विशिष्ट इवेंट्स सुनें:
- `RequestInfoEvent` - मानव इनपुट की आवश्यकता है (वर्कफ़्लो रुका हुआ)
- `WorkflowOutputEvent` - अंतिम परिणाम उपलब्ध है (वर्कफ़्लो पूरा)
- `WorkflowStatusEvent` - स्थिति परिवर्तन (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS, आदि)

#### 4. **@handler के साथ कस्टम एक्सीक्यूटर्स**
`DecisionManager` दिखाता है कि कैसे एक्सीक्यूटर्स बनाए जाएं जो:
- `@handler` डेकोरेटर का उपयोग करके विधियों को वर्कफ़्लो चरणों के रूप में उजागर करते हैं
- टाइप किए गए संदेश प्राप्त करते हैं (जैसे, `RequestResponse[HumanFeedbackRequest, str]`)
- वर्कफ़्लो को अन्य एक्सीक्यूटर्स को संदेश भेजकर रूट करते हैं
- `WorkflowContext` के माध्यम से संदर्भ तक पहुंचते हैं

#### 5. **मानव निर्णयों के साथ कंडीशनल रूटिंग**
आप ऐसी कंडीशन फंक्शन्स बना सकते हैं जो मानव प्रतिक्रियाओं का मूल्यांकन करती हैं:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 वास्तविक दुनिया के अनुप्रयोग:

1. **अनुमोदन वर्कफ़्लो**
   - खर्च रिपोर्ट्स को प्रोसेस करने से पहले प्रबंधक की मंजूरी प्राप्त करें
   - स्वचालित ईमेल भेजने से पहले मानव समीक्षा की आवश्यकता हो
   - उच्च-मूल्य लेनदेन को निष्पादित करने से पहले पुष्टि करें

2. **सामग्री मॉडरेशन**
   - संदिग्ध सामग्री को मानव समीक्षा के लिए चिह्नित करें
   - किनारे के मामलों में अंतिम निर्णय लेने के लिए मॉडरेटर से पूछें
   - जब AI का आत्मविश्वास कम हो तो इसे मानव तक बढ़ाएं

3. **ग्राहक सेवा**
   - AI को स्वचालित रूप से सामान्य प्रश्नों को संभालने दें
   - जटिल मुद्दों को मानव एजेंट्स तक बढ़ाएं
   - ग्राहक से पूछें कि क्या वे किसी मानव से बात करना चाहते हैं

4. **डेटा प्रोसेसिंग**
   - अस्पष्ट डेटा प्रविष्टियों को हल करने के लिए मानव से पूछें
   - अस्पष्ट दस्तावेजों की AI व्याख्याओं की पुष्टि करें
   - उपयोगकर्ताओं को कई वैध व्याख्याओं में से चुनने दें

5. **सुरक्षा-महत्वपूर्ण सिस्टम**
   - अपरिवर्तनीय क्रियाओं से पहले मानव पुष्टि की आवश्यकता हो
   - संवेदनशील डेटा तक पहुंचने से पहले अनुमोदन प्राप्त करें
   - विनियमित उद्योगों (स्वास्थ्य सेवा, वित्त) में निर्णयों की पुष्टि करें

6. **इंटरएक्टिव एजेंट्स**
   - बातचीत करने वाले बॉट्स बनाएं जो फॉलो-अप प्रश्न पूछते हैं
   - उपयोगकर्ताओं को जटिल प्रक्रियाओं के माध्यम से मार्गदर्शन करने वाले विज़ार्ड बनाएं
   - ऐसे एजेंट्स डिज़ाइन करें जो मानव के साथ कदम-दर-कदम सहयोग करते हैं

### 🔄 तुलना: मानव-इन-द-लूप के साथ बनाम बिना

| विशेषता | कंडीशनल वर्कफ़्लो | मानव-इन-द-लूप वर्कफ़्लो |
|---------|---------------------|---------------------------|
| **निष्पादन** | एकल `workflow.run()` | `run_stream()` + `send_responses_streaming()` के साथ लूप |
| **उपयोगकर्ता इनपुट** | कोई नहीं (पूरी तरह स्वचालित) | `input()` या UI के माध्यम से इंटरएक्टिव प्रॉम्प्ट्स |
| **घटक** | एजेंट्स + एक्सीक्यूटर्स | + RequestInfoExecutor + DecisionManager |
| **इवेंट्स** | केवल AgentExecutorResponse | RequestInfoEvent, WorkflowOutputEvent, आदि |
| **रोकना** | कोई रोक नहीं | RequestInfoExecutor पर वर्कफ़्लो रुकता है |
| **मानव नियंत्रण** | कोई मानव नियंत्रण नहीं | मानव प्रमुख निर्णय लेते हैं |
| **उपयोग का मामला** | स्वचालन | सहयोग और निगरानी |

### 🚀 उन्नत पैटर्न:

#### कई मानव निर्णय बिंदु
आप एक ही वर्कफ़्लो में कई `RequestInfoExecutor` नोड्स रख सकते हैं:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### टाइमआउट हैंडलिंग
मानव प्रतिक्रियाओं के लिए टाइमआउट लागू करें:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### रिच UI इंटीग्रेशन
`input()` के बजाय, वेब UI, Slack, Teams, आदि के साथ इंटीग्रेट करें:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### कंडीशनल मानव-इन-द-लूप
केवल विशिष्ट परिस्थितियों में मानव इनपुट के लिए पूछें:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ सर्वोत्तम प्रथाएँ:

1. **हमेशा RequestInfoMessage को सबक्लास करें**
   - टाइप सेफ्टी और वैलिडेशन प्रदान करता है
   - UI रेंडरिंग के लिए समृद्ध संदर्भ सक्षम करता है
   - प्रत्येक अनुरोध प्रकार के इरादे को स्पष्ट करता है

2. **वर्णनात्मक प्रॉम्प्ट्स का उपयोग करें**
   - आप जो पूछ रहे हैं उसका संदर्भ शामिल करें
   - प्रत्येक विकल्प के परिणामों को समझाएं
   - प्रश्नों को सरल और स्पष्ट रखें

3. **अप्रत्याशित इनपुट को संभालें**
   - उपयोगकर्ता प्रतिक्रियाओं को मान्य करें
   - अमान्य इनपुट के लिए डिफॉल्ट्स प्रदान करें
   - स्पष्ट त्रुटि संदेश दें

4. **अनुरोध IDs को ट्रैक करें**
   - request_id और प्रतिक्रियाओं के बीच संबंध का उपयोग करें
   - मैन्युअल रूप से स्थिति प्रबंधित करने की कोशिश न करें

5. **नॉन-ब्लॉकिंग डिज़ाइन करें**
   - इनपुट की प्रतीक्षा करते हुए थ्रेड्स को ब्लॉक न करें
   - पूरे में async पैटर्न का उपयोग करें
   - समवर्ती वर्कफ़्लो इंस्टेंस का समर्थन करें

### 📚 संबंधित अवधारणाएँ:

- **एजेंट मिडलवेयर** - एजेंट कॉल्स को इंटरसेप्ट करें (पिछला नोटबुक)
- **वर्कफ़्लो स्टेट मैनेजमेंट** - रन के बीच वर्कफ़्लो स्थिति को बनाए रखें
- **मल्टी-एजेंट सहयोग** - मानव-इन-द-लूप को एजेंट टीमों के साथ मिलाएं
- **इवेंट-ड्रिवन आर्किटेक्चर** - इवेंट्स के साथ प्रतिक्रियात्मक सिस्टम बनाएं

---

### 🎓 बधाई हो!

आपने Microsoft Agent Framework के साथ मानव-इन-द-लूप वर्कफ़्लो में महारत हासिल कर ली है! अब आप जानते हैं कि:
- ✅ वर्कफ़्लो को मानव इनपुट एकत्र करने के लिए रोकें
- ✅ RequestInfoExecutor और RequestInfoMessage का उपयोग करें
- ✅ इवेंट्स के साथ स्ट्रीमिंग निष्पादन को संभालें
- ✅ @handler के साथ कस्टम एक्सीक्यूटर्स बनाएं
- ✅ मानव निर्णयों के आधार पर वर्कफ़्लो को रूट करें
- ✅ इंटरएक्टिव AI एजेंट्स बनाएं जो मानव के साथ सहयोग करते हैं

**यह भरोसेमंद, नियंत्रित AI सिस्टम बनाने के लिए एक महत्वपूर्ण पैटर्न है!** 🚀



---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता के लिए प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में दस्तावेज़ को आधिकारिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।
